In [2]:
import pandas as pd
from datetime import datetime

In [3]:
data = pd.read_csv('../dataset/power-laws-forecasting-energy-consumption-holidays.csv', delimiter=';', usecols=lambda column: column != 'Unnamed: 0')
data

,Date,Holiday,SiteId
0,2016-02-15,Washington's Birthday,1
1,2017-05-29,Memorial Day,1
2,2017-11-23,Thanksgiving Day,1
3,2017-12-29,New Years Eve Shift,1
4,2017-12-31,New Years Eve,1
...,...,...,...
8382,2015-12-26,Boxing Day,303
8383,2016-05-01,International Workers' Day,304
8384,2015-04-25,Liberation Day,304
8385,2016-03-28,Easter Monday,305


In [4]:
data = pd.read_csv('../dataset/power-laws-forecasting-energy-consumption-metadata.csv', delimiter=';', usecols=lambda column: column != 'Unnamed: 0')
data

,SiteId,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,WednesdayIsDayOff,ThursdayIsDayOff,FridayIsDayOff,SaturdayIsDayOff,SundayIsDayOff
0,207,7964.873347,30.0,18.0,False,False,False,False,False,True,True
1,7,15168.125971,30.0,18.0,False,False,False,False,False,True,True
2,74,424.340663,15.0,18.0,False,False,False,False,False,True,True
3,239,1164.822636,15.0,18.0,False,False,False,False,False,True,True
4,274,1468.246690,5.0,18.0,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...
262,192,11188.881545,15.0,18.0,False,False,False,False,False,True,True
263,58,1149.050606,15.0,18.0,False,False,False,False,False,True,True
264,123,5470.205018,15.0,18.0,False,False,False,False,False,True,True
265,122,6843.612340,15.0,18.0,False,False,False,False,False,True,True


In [5]:
data = pd.read_csv('../dataset/power-laws-forecasting-energy-consumption-submission-forecast-period.csv', delimiter=';', usecols=lambda column: column != 'Unnamed: 0')
data

,ForecastId,ForecastPeriodNS
0,123,900000000000
1,264,900000000000
2,596,900000000000
3,914,900000000000
4,1053,900000000000
...,...,...
6969,6387,900000000000
6970,6487,3600000000000
6971,6569,900000000000
6972,6777,900000000000


In [6]:
data = pd.read_csv('../dataset/power-laws-forecasting-energy-consumption-test-data.csv', delimiter=';', usecols=lambda column: column != 'Unnamed: 0')
data

,obs_id,SiteId,Timestamp,ForecastId,Value
0,323604,235,2014-01-02T19:00:00+00:00,5004,157265.446409
1,2813181,235,2014-01-02T22:00:00+00:00,5004,155498.418922
2,4006999,235,2014-01-02T22:45:00+00:00,5004,155498.418922
3,106973,235,2014-01-03T00:30:00+00:00,5004,157265.446409
4,6793052,235,2014-01-13T14:15:00+00:00,5005,91885.429363
...,...,...,...,...,...
1309171,268524,261,2014-03-24T03:00:00+00:00,5487,38545.410018
1309172,3203969,261,2014-03-24T12:30:00+00:00,5487,48937.146780
1309173,7721495,261,2014-03-24T22:00:00+00:00,5487,NaN
1309174,5696668,261,2014-03-24T23:30:00+00:00,5487,NaN


In [7]:
data = pd.read_csv('../dataset/power-laws-forecasting-energy-consumption-training-data.csv', delimiter=';', usecols=lambda column: column != 'Unnamed: 0')
data

,obs_id,SiteId,Timestamp,ForecastId,Value
0,4852050,42,2016-10-18T02:45:00+00:00,1087,26397.049623
1,1638923,42,2016-10-18T11:45:00+00:00,1087,42958.364641
2,5748910,42,2016-10-18T20:45:00+00:00,1087,27096.919666
3,38199,42,2016-10-20T10:45:00+00:00,1087,50211.408087
4,1338204,42,2016-10-20T18:45:00+00:00,1087,50503.305105
...,...,...,...,...,...
6559825,1127574,300,2017-09-22T18:45:00+00:00,6719,7740.955427
6559826,4695712,300,2017-09-23T17:45:00+00:00,6719,7133.180234
6559827,978979,300,2017-09-24T22:45:00+00:00,6719,7339.789365
6559828,6317358,300,2017-09-25T08:45:00+00:00,6719,18873.744081


In [8]:
data = pd.read_csv('../dataset/power-laws-forecasting-energy-consumption-weather.csv', delimiter=';', usecols=lambda column: column != 'Unnamed: 0')
data

,Timestamp,Temperature,Distance,SiteId
0,2017-03-03T19:00:00+00:00,10.6,27.489346,51
1,2017-03-03T19:20:00+00:00,11.0,28.663082,51
2,2017-03-03T20:00:00+00:00,6.3,28.307039,51
3,2017-03-03T21:55:00+00:00,10.0,29.797449,51
4,2017-03-03T23:00:00+00:00,5.4,28.307039,51
...,...,...,...,...
3957030,2016-09-11T11:00:00+00:00,25.9,28.307039,51
3957031,2016-09-11T11:20:00+00:00,27.0,27.489346,51
3957032,2016-09-11T12:00:00+00:00,27.1,28.307039,51
3957033,2016-09-11T15:50:00+00:00,28.0,27.489346,51
